In [27]:
# Imports
import polars as pl
import altair as alt
from vega_datasets import data
import pandas as pd

In [143]:
def china_solar_exports_df():
    df = pl.read_csv("data/mart_solar_exports_full_release_monthly.csv")
    
    # This is the correct pattern
    return (
        df.with_columns(
            pl.col("Date").str.to_date()  # 1. Update the Date column
        )
        .filter(
            # 2. THEN, filter the entire DataFrame
            pl.col("Area").is_in([
                "Other Oceania", 
                "Other Africa", 
                "Other Latin America"
            ]).not_() # (or use the ~ operator)
        )
    )

In [144]:
def china_solar_exports_countries_df():
    return china_solar_exports_df().filter(
        pl.col("Area type") == "Country or economy"
    )

china_solar_exports_countries_df()

Area,Date,Area type,Region,Commodity code,Commodity type,Amount (items),Amount (kg),Amount (USD),Commodity price,Price unit,Capacity (MW),Cumulative capacity (MW)
str,date,str,str,i64,str,i64,i64,i64,f64,str,f64,f64
"""Afghanistan""",0017-01-01,"""Country or economy""","""Asia""",85414020,"""Panels""",49433,333324,1663024,0.38,"""USD/W""",4.38,4.38
"""Afghanistan""",0017-01-02,"""Country or economy""","""Asia""",85414020,"""Panels""",5452,82456,337723,0.38,"""USD/W""",0.89,5.27
"""Afghanistan""",0017-01-03,"""Country or economy""","""Asia""",85414020,"""Panels""",51403,940507,3477201,0.38,"""USD/W""",9.15,14.42
"""Afghanistan""",0017-01-04,"""Country or economy""","""Asia""",85414020,"""Panels""",100875,1306053,4783758,0.38,"""USD/W""",12.59,27.0
"""Afghanistan""",0017-01-05,"""Country or economy""","""Asia""",85414020,"""Panels""",67325,786843,2823671,0.38,"""USD/W""",7.43,34.44
…,…,…,…,…,…,…,…,…,…,…,…,…
"""Zimbabwe""",0025-01-06,"""Country or economy""","""Africa""",85414300,"""Panels""",15891,351968,586375,0.09,"""USD/W""",6.18,470.44
"""Zimbabwe""",0025-01-07,"""Country or economy""","""Africa""",85414200,"""Cells""",1611,26079,48306,0.04,"""USD/W""",1.32,21.78
"""Zimbabwe""",0025-01-07,"""Country or economy""","""Africa""",85414300,"""Panels""",36213,733391,1441831,0.09,"""USD/W""",15.22,485.66


In [10]:
def china_solar_exports_regions_df(region: str):
    df = china_solar_exports_df().filter(pl.col("Area type") == "Region")

    if region == "world":
        return df.filter(pl.col("Area") == "World")
    elif region == "regions":
          return df.filter(pl.col("Area") != "World")
    else:
        return df.filter(pl.col("Area") == region)
    
def china_solar_exports_monthly(region: str):
    df = china_solar_exports_regions_df(region)

    result = (
        df.group_by(["Area", "Date", "Commodity type"]) 
        .agg(pl.col("Capacity (MW)").sum().alias("Total Capacity (MW)"))
        .rename({"Area": "Region"})
        .sort(["Region", "Date", "Commodity type"])
    )

    return result

def china_solar_exports_monthly_by_region(region: str):
    
    # Get the detailed data (with commodity types)
    detailed_df = china_solar_exports_monthly(region)
    
    # Add a final aggregation to sum up the commodity types
    region_agg_df = (
        detailed_df.group_by(["Region", "Date"])
        .agg(pl.col("Total Capacity (MW)").sum()) # Sums the commodities
        .sort(["Region", "Date"])
    )
    return region_agg_df


In [11]:
china_solar_exports_monthly_by_region("regions") 

Region,Date,Total Capacity (MW)
str,date,f64
"""ASEAN""",2017-01-01,32.31
"""ASEAN""",2017-02-01,29.09
"""ASEAN""",2017-03-01,49.94
"""ASEAN""",2017-04-01,48.92
"""ASEAN""",2017-05-01,44.74
…,…,…
"""Oceania""",2025-04-01,601.02
"""Oceania""",2025-05-01,731.41
"""Oceania""",2025-06-01,698.98


In [12]:
def plot_world_exports(color: str):
    """
    Creates a bar chart of China's total monthly solar exports to the world.
    """
    chart = (
        alt.Chart(china_solar_exports_monthly("world"))
        .mark_bar(color=color) 
        .encode(
            x=alt.X(
                "Date:T",
                title="Year",
                axis=alt.Axis(
                    format="%Y",
                    tickCount="year",
                )
            ),
            y=alt.Y("Total Capacity (MW):Q", title="Total Capacity (MW)"),
            tooltip=[
                alt.Tooltip("Date:T", title="Month"),
                alt.Tooltip("Total Capacity (MW):Q", title="Total Capacity (MW)")
            ]
        )
        .properties(width=700, height=400, title="China Solar Exports - By Monthly Capacity (World)")
    )
    
    return chart

plot_world_exports("#C02C38").save("images_plots/plot_world_exports_red.png", scale_factor=2.0)

In [13]:
plot_world_exports("#E77C8E").save("images_plots/plot_world_exports_pinkish.png", scale_factor=2.0)

In [14]:
def plot_world_exports_stacked_bar():
    """
    Creates a stacked bar chart of China's monthly solar exports
    by technology, with a custom color scheme.
    """
    
    # Load data
    df_world_details = china_solar_exports_monthly("world")

    domain_ = ["Panels", "Cells", "Wafers"] 
    range_ = ['#BACF65', '#41AE3C', '#ADD5A2']

    # Create the stacked bar chart
    stacked_bar_chart = (
        alt.Chart(df_world_details)
        .mark_bar() 
        .encode(
            # X-axis: Time (Date)
            x=alt.X(
                "Date:T",
                title="Year",
                axis=alt.Axis(format="%Y") 
            ),
            
            # Y-axis: Total capacity, stacked to zero
            y=alt.Y(
                "Total Capacity (MW):Q", 
                title="Total Capacity (MW)", 
                stack="zero"
            ),
            
            # Color: Set the custom domain and range
            color=alt.Color(
                "Commodity type:N", 
                title="Technology",
                scale=alt.Scale(domain=domain_, range=range_)
            ),
            
            # Tooltip: Show details on hover
            tooltip=[
                alt.Tooltip("Date:T", title="Date", format="%Y-%m"),
                alt.Tooltip("Commodity type:N", title="Technology"),
                alt.Tooltip("Total Capacity (MW):Q", title="Monthly Capacity (MW)", format=",")
            ]
        )
        .properties(
            title="China's Monthly Solar Exports by Technology (World)",
            width=750,
            height=400
        )
        .interactive() 
    )
    
    return stacked_bar_chart


plot_world_exports_stacked_bar().save("images_plots/monthly_exports_by_technology.png", scale_factor=2.0)

In [15]:
def plot_world_exports_stacked_bar():
    """
    Creates a stacked bar chart of China's monthly solar exports
    by technology, with a custom color scheme.
    """
    
    # Load data
    df_world_details = china_solar_exports_monthly("world")

    domain_ = ["Panels", "Cells", "Wafers"] 
    range_ = ['#FBA414', '#12AA9C', '#5A1216']

    # Create the stacked BAR chart
    stacked_bar_chart = (
        alt.Chart(df_world_details)
        .mark_bar() 
        .encode(
            # X-axis: Time (Date)
            x=alt.X(
                "Date:T",
                title="Year",
                axis=alt.Axis(format="%Y") 
            ),
            
            # Y-axis: Total capacity, stacked to zero
            y=alt.Y(
                "Total Capacity (MW):Q", 
                title="Total Capacity (MW)", 
                stack="zero"
            ),
            
            # Color: Set the custom domain and range
            color=alt.Color(
                "Commodity type:N", 
                title="Technology",
                scale=alt.Scale(domain=domain_, range=range_)
            ),
            
            # Tooltip: Show details on hover
            tooltip=[
                alt.Tooltip("Date:T", title="Date", format="%Y-%m"),
                alt.Tooltip("Commodity type:N", title="Technology"),
                alt.Tooltip("Total Capacity (MW):Q", title="Monthly Capacity (MW)", format=",")
            ]
        )
        .properties(
            title="China's Monthly Solar Exports by Technology (World)",
            width=750,
            height=400
        )
        .interactive()
    )
    
    return stacked_bar_chart

plot_world_exports_stacked_bar().save("images_plots/monthly_exports_by_technology_colorful.png", scale_factor=2.0)

In [16]:
def worl_exports_stacked_area():
    df_world_details = china_solar_exports_monthly("world")

    domain_ = ["Panels", "Cells", "Wafers"] 
    range_ = ['#FBA414', '#12AA9C', '#5A1216']

    # Create the stacked area chart
    stacked_area_chart = (
        alt.Chart(df_world_details)
        .mark_area()
        .encode(
            # X-axis: Time (Date)
            x=alt.X(
                "Date:T",
                title="Year",
                axis=alt.Axis(format="%Y", tickCount="year")
            ),
            
            # Y-axis: Total capacity, stacked to zero
            y=alt.Y(
                "Total Capacity (MW):Q", 
                title="Total Capacity (MW)", 
                stack="zero"
            ),
            
            # Color: The different technology types
            color=alt.Color(
                "Commodity type:N", 
                title="Technology",
                scale=alt.Scale(domain=domain_, range=range_)
            ),
            
            # Tooltip: Show details on hover
            tooltip=[
                alt.Tooltip("Date:T", title="Date", format="%Y-%m"),
                alt.Tooltip("Commodity type:N", title="Technology"),
                alt.Tooltip("Total Capacity (MW):Q", title="Monthly Capacity (MW)", format=",")
            ]
        )
        .properties(
            title="China's Monthly Solar Exports by Technology (World)",
            width=750,
            height=400
        )
        .interactive()
    )

    return stacked_area_chart

worl_exports_stacked_area().save("images_plots/exports_by_tech.png", scale_factor=2.0)

In [17]:
df_world_details = china_solar_exports_monthly("world")

domain_ = ["Panels", "Cells", "Wafers"] 
range_ = ['#FCA106', '#F8E0B0', '#826B48']

# Create the stacked area chart
stacked_area_chart = (
    alt.Chart(df_world_details)
    .mark_area()
    .encode(
        # X-axis: Time (Date)
        x=alt.X(
            "Date:T",
            title="Year",
            axis=alt.Axis(format="%Y", tickCount="year")
        ),
        
        # Y-axis: Total capacity, stacked to zero
        y=alt.Y(
            "Total Capacity (MW):Q", 
            title="Total Capacity (MW)", 
            stack="zero"
        ),
        
        # Color: The different technology types
        color=alt.Color(
            "Commodity type:N", 
            title="Technology",
            scale=alt.Scale(domain=domain_, range=range_)
        ),
        
        # Tooltip: Show details on hover
        tooltip=[
            alt.Tooltip("Date:T", title="Date", format="%Y-%m"),
            alt.Tooltip("Commodity type:N", title="Technology"),
            alt.Tooltip("Total Capacity (MW):Q", title="Monthly Capacity (MW)", format=",")
        ]
    )
    .properties(
        title="China's Monthly Solar Exports by Technology (World)",
        width=750,
        height=400
    )
    .interactive()
)

stacked_area_chart

alt.Chart(...)

In [18]:
def plot_regional_exports_by_month():
    """
    Creates a line chart of China's monthly solar exports for each region,
    with points and labels at the end of each line.
    """
    
    # Create the base chart
    base = alt.Chart(china_solar_exports_monthly_by_region("regions")).encode(
        x=alt.X(
            "Date:T",
            title="Year",
            axis=alt.Axis(format="%Y", tickCount="year")
        ),
        y=alt.Y("Total Capacity (MW):Q", title="Total Capacity (MW)"),
        color=alt.Color("Region:N", title="Region")
    )

    # Line chart layer
    lines = base.mark_line().encode(
        tooltip=["Region", "Date:T", "Total Capacity (MW):Q"]
    )

    # Points layer for the last value of each region
    last_points = (
    base
    .transform_window(
        rank='rank()', sort=[alt.SortField('Date', order='descending')]
    )
    .transform_filter('datum.rank == 1')
    .mark_point(filled=True)
    .encode(
        size=alt.condition(
            alt.datum.Region == "World",
            alt.value(100),
            alt.value(60)
        )))

    # Labels layer for the last point
    labels_bold = (
    base
    .transform_window(
        rank='rank()', sort=[alt.SortField('Date', order='descending')]
    )
    .transform_filter('datum.rank == 1')
    .transform_filter(alt.datum.Region == "World")
    .mark_text(align="left", dx=5, fontWeight="bold", clip=False)
    .encode(
        text="Region:N"
    ))

    # Combine all the layers into a single chart
    chart = (
        (lines + last_points + labels_bold)
        .properties(
            width=750,
            height=400,
            title="China Solar Exports - Monthly Capacity by Region"
        )
        .configure_axis(
            grid=False
        )
    )

    return chart


plot_regional_exports_by_month().save("images_plots/exports_by_regions.png", scale_factor=2.0)

In [19]:
def plot_regional_exports_with_world():
    """
    Creates a line chart of regional exports and a specially-styled
    line for the 'World' total, with points and labels at the end.
    """
    
    regions_to_remove = ["EU", "G20", "G7", "OECD", "ASEAN"]
    
    # --- FIX 1: Add "World" to the domain and range ---
    domain_list = ["World", "Africa", "Asia", "Europe", "Latin America and Caribbean", "Middle East", "North America", "Oceania"]
    
    # Added "#000000" (black) for "World" and fixed typo '##5F3C4F'
    color_range = ["#C3272B", "#D0957E","#F7D881","#006796","#5F3C4F","#ECB16B","#6ABA92","#8A7F8D"]

    # 1. Call your function
    df_regions = china_solar_exports_monthly_by_region("regions").filter(
        pl.col("Region").is_in(regions_to_remove).not_()
    )

    # 2. Load the world data
    world_df = china_solar_exports_monthly_by_region("world")

    # 3. Combine them into a single DataFrame
    combined_df = pl.concat([df_regions, world_df])

    # --- FIX 2: Remove 'legend=None' ---
    color_encoding = alt.Color(
            "Region:N",
            title="Region",
            # legend=None,  <-- This line was removed to show the legend
            scale=alt.Scale(domain=domain_list, range=color_range)
        )

    # Create the base chart
    base = alt.Chart(combined_df).encode(
        x=alt.X(
            "Date:T",
            title="Year",
            axis=alt.Axis(format="%Y", tickCount="year")
        ),
        y=alt.Y("Total Capacity (MW):Q", title="Total Capacity (MW)"),
        color=color_encoding
    )

    # --- MODIFIED Line chart ---
    lines = base.mark_line().encode(
        tooltip=["Region", "Date", "Total Capacity (MW):Q"],
        strokeWidth=alt.condition(
            alt.datum.Region == "World",
            alt.value(3.5),
            alt.value(1.5)
        ),
        strokeDash=alt.condition(
            alt.datum.Region == "World",
            alt.value([5, 3]),
            alt.value([1, 0])
        )
    )

    # --- MODIFIED Points ---
    # --- Points (use the working window logic) ---
    last_points = (
        base
        .transform_window(
            rank='rank()', sort=[alt.SortField('Date', order='descending')]
        )
        .transform_filter('datum.rank == 1')
        .mark_point(filled=True)
        .encode(
            size=alt.condition(
                alt.datum.Region == "World",
                alt.value(100),
                alt.value(60)
            )
        )
    )

    # --- Labels for regions except World ---
    labels_normal = (
        alt.Chart(combined_df)
        .transform_window(
            rank='rank()', sort=[alt.SortField('Date', order='descending')]
        )
        .transform_filter('datum.rank == 1')
        .transform_filter(alt.datum.Region != "World")
        .mark_text(align="left", dx=5, fontWeight="normal")
        .encode(
            x="Date:T",
            y="Total Capacity (MW):Q",
            text="Region:N",
            color=alt.value("black")  # break color inheritance → fixes legend issue
        )
    )

    # --- Label for World ---
    labels_bold = (
        alt.Chart(combined_df)
        .transform_window(
            rank='rank()', sort=[alt.SortField('Date', order='descending')]
        )
        .transform_filter('datum.rank == 1')
        .transform_filter(alt.datum.Region == "World")
        .mark_text(align="left", dx=5, fontWeight="bold")
        .encode(
            x="Date:T",
            y="Total Capacity (MW):Q",
            text="Region:N",
            color=alt.value("black")
        )
    )

    # --- Combine all layers ---
    styled_chart = (
        # Add both label layers
        (lines + last_points + labels_normal + labels_bold)
        .properties(
            width=750,
            height=400,
            title="China Solar Exports - Monthly Capacity by Region (with World Total)"
        )
        .configure_axis(
            grid=False
        )
    )

    return styled_chart


plot_regional_exports_with_world().save("images_plots/exports_by_region_plus_world.png", scale_factor=2.0)

In [20]:
# Improved by Gemini after prompt "create a bar chart, grouped by regions and 
# with one bar per technology in an dataframe using altair"

def plot_regional_exports_by_technology():
    """
    Creates a grouped bar chart showing the total export capacity for each
    solar technology, faceted by region.
    """
    
    # Load data
    df_regions_details = china_solar_exports_monthly("regions")
    domain_list=['Panels', 'Cells', 'Wafers']
    color_range=['#FFCF63', '#A52A2A', '#DECBC6']

    # Create the Grouped Bar Chart
    grouped_bar_chart = (
        alt.Chart(df_regions_details)
        .mark_bar()
        .encode(
            # Group the technologies on x-axis
            x=alt.X("Commodity type:N", title=None, axis=None),
            
            # y-axis is the total sum for that technology
            y=alt.Y(
                "sum(Total Capacity (MW)):Q",
                title="Total Capacity (MW)"
            ),
            
            # Color the bars by the technology type with a custom scale
            color=alt.Color(
                "Commodity type:N",
                title="Technology",
                scale=alt.Scale(domain=domain_list, range=color_range)
            ),
            
            # Column for each region to group the bars
            column=alt.Column(
                "Region:N",
                title="Region",
                header=alt.Header(titleOrient="bottom", labelOrient="bottom", titlePadding=5),
            ),
            
            # tooltip
            tooltip=[
                "Region:N",
                "Commodity type:N",
                alt.Tooltip(
                    "sum(Total Capacity (MW)):Q",
                    title="Total Capacity (MW)",
                    format=","
                ),
            ],
        )
        .properties(
            title="Total Solar Exports by Technology and Region",
            width=alt.Step(25),  # Sets the width of each *individual bar*
            height=300
        )
        .configure_facet(
            spacing=15  # Adds a little space between each region's group
        )
        .configure_view(
            stroke=None  # Removes outer border
        )
    )

    return grouped_bar_chart


plot_regional_exports_by_technology().save("images_plots/exports_by_tech_region.png", scale_factor=2.0)

In [21]:
def plot_regional_composition_facet():
    """
    Creates a faceted 100% stacked bar chart showing the composition
    of solar exports for each region.
    """

    domain_list=['Panels', 'Cells', 'Wafers']
    color_range=['#FFCF63', '#A52A2A', '#DECBC6']

    # Load data
    df_regions_details = china_solar_exports_monthly("regions")

    # Create the faceted 100% stacked bar chart
    wrapped_normalized_bar_chart = (
        alt.Chart(df_regions_details)
        .mark_bar()
        .encode(
            # Y-axis: Sum of capacity, normalized to 100%
            y=alt.Y(
                "sum(Total Capacity (MW)):Q",
                title="Share of Exports",
                stack="normalize",  # 100% stack
                axis=alt.Axis(format="%") # Format axis as percentage
            ),
            
            # Color
            color=alt.Color(
                "Commodity type:N",
                title="Technology",
                scale=alt.Scale(domain=domain_list, range=color_range)
            ),

            # Tooltip
            tooltip=[
                "Region:N",
                "Commodity type:N",
                alt.Tooltip(
                    "sum(Total Capacity (MW)):Q",
                    title="Total Capacity (MW)",
                    format=","
                )
            ],

            # Facet
            facet=alt.Facet(
                "Region:N",
                columns=4  
            )
        )
        .properties(
            title="Composition of China's Solar Exports by Region (All Time)",
            # width/height for each individual chart in the grid
            width=150,
            height=300
        )
    )

    return wrapped_normalized_bar_chart


plot_regional_composition_facet().save("images_plots/monthly_exports_by_tech_region_canvas.png", scale_factor=2.0)

In [44]:
def cofi_df():
    techs = ["solar csp", "solar pv"]
    return (
        pl.read_excel("/Users/chemagalvez/Documents/COFI_Database_Version_October_2023.xlsx")
        .filter(pl.col("primary_fuel").str.to_lowercase().is_in(techs))
        .select([
            "unique_id",
            "power_plant_name",
            "country",
            "country_iso3c",
            "region",
            "subregion",
            "location_id.x",
            "equity_investment_year",
            "installed_capacity",
            "total_investment_amount",
            "duplicate_power_plant",
            "investment_type"
        ])
    )


In [45]:
cofi_df()

Could not determine dtype for column 39, falling back to string
Could not determine dtype for column 40, falling back to string
Could not determine dtype for column 42, falling back to string


unique_id,power_plant_name,country,country_iso3c,region,subregion,location_id.x,equity_investment_year,installed_capacity,total_investment_amount,duplicate_power_plant,investment_type
str,str,str,str,str,str,i64,i64,f64,f64,str,str
"""ARE2021001""","""AL DHAFRA PV2""","""United Arab Emirates""","""ARE""","""Asia""","""Western Asia""",1134119,2020,1584.0,288.753636,"""N""","""Both"""
"""ARE2021004""","""MOHAMMED AL MAKTOUM-IV""","""United Arab Emirates""","""ARE""","""Asia""","""Western Asia""",1118444,2019,700.0,1182.2,"""N""","""Both"""
"""ARE2021005""","""MOHAMMED AL MAKTOUM-IV""","""United Arab Emirates""","""ARE""","""Asia""","""Western Asia""",1118444,null,null,252.6,"""Y""","""Debt"""
"""ARE2022007""","""MOHAMMED AL MAKTOUM-IV""","""United Arab Emirates""","""ARE""","""Asia""","""Western Asia""",1118444,2019,250.0,196.74,"""N""","""Both"""
"""ARE2022008""","""MOHAMMED AL MAKTOUM-IV""","""United Arab Emirates""","""ARE""","""Asia""","""Western Asia""",1118444,null,null,315.76,"""Y""","""Debt"""
…,…,…,…,…,…,…,…,…,…,…,…
"""URY2022004""","""Sky Solar's Uruguay PV portfol…","""Uruguay""","""URY""","""Americas""","""Latin America and the Caribbea…",null,null,69.9,19.3,"""N""","""Debt"""
"""VNM2021019""",null,"""Vietnam""","""VNM""","""Asia""","""South-eastern Asia""",null,2016,40.0,null,"""N""","""Equity"""
"""ZAF2021009""","""DYASON'S KLIP""","""South Africa""","""ZAF""","""Africa""","""Sub-Saharan Africa""",1107416,null,75.0,61.721665,"""N""","""Debt"""


In [48]:
def cofi_df():
    techs = ["solar csp", "solar pv"]
    return (
        pl.read_excel("/Users/chemagalvez/Documents/COFI_Database_Version_October_2023.xlsx")
        .filter(pl.col("primary_fuel").str.to_lowercase().is_in(techs))
        .select([
            "country",
            "country_iso3c",
            "region",
            "location_id.x",
            "installed_capacity",
            pl.col("total_investment_amount").cast(pl.Float64).alias("total_investment_amount"),
            pl.col("primary_fuel").str.to_lowercase().alias("primary_fuel")
        ])
    )
cofi_df()

Could not determine dtype for column 39, falling back to string
Could not determine dtype for column 40, falling back to string
Could not determine dtype for column 42, falling back to string


country,country_iso3c,region,location_id.x,installed_capacity,total_investment_amount,primary_fuel
str,str,str,i64,f64,f64,str
"""United Arab Emirates""","""ARE""","""Asia""",1134119,1584.0,288.753636,"""solar pv"""
"""United Arab Emirates""","""ARE""","""Asia""",1118444,700.0,1182.2,"""solar csp"""
"""United Arab Emirates""","""ARE""","""Asia""",1118444,null,252.6,"""solar csp"""
"""United Arab Emirates""","""ARE""","""Asia""",1118444,250.0,196.74,"""solar pv"""
"""United Arab Emirates""","""ARE""","""Asia""",1118444,null,315.76,"""solar pv"""
…,…,…,…,…,…,…
"""Uruguay""","""URY""","""Americas""",null,69.9,19.3,"""solar pv"""
"""Vietnam""","""VNM""","""Asia""",null,40.0,null,"""solar pv"""
"""South Africa""","""ZAF""","""Africa""",1107416,75.0,61.721665,"""solar pv"""


In [51]:
# Plant level (avoid double counting)
plant_level = (
    cofi_df()
    .group_by(["country", "country_iso3c", "region", "location_id.x"])
    .agg(pl.col("total_investment_amount").max().alias("plant_total_investment"))
)

# Country totals
country_totals = (
    plant_level.group_by(["country", "country_iso3c", "region"])
    .agg(pl.col("plant_total_investment").sum().alias("total_investment_musd"))
    .sort("total_investment_musd", descending=True)
)

df_countries = country_totals.to_pandas()

# Merge plant-level back with capacity + fuel for visualization #3
df_plants = (
    plant_level.join(
        cofi_df().select(["location_id.x", "installed_capacity", "primary_fuel"]),
        on="location_id.x",
        how="left"
    )
    .to_pandas()
)


Could not determine dtype for column 39, falling back to string
Could not determine dtype for column 40, falling back to string
Could not determine dtype for column 42, falling back to string


Could not determine dtype for column 39, falling back to string
Could not determine dtype for column 40, falling back to string
Could not determine dtype for column 42, falling back to string


In [52]:


TOP_N = 25
df_top = df_countries.head(TOP_N)

base = alt.Chart(df_top).encode(
    y=alt.Y('country:N', sort='-x', title=None)
)

lines = base.mark_rule().encode(
    x=alt.X('total_investment_musd:Q', title='Total Investment (USD Millions)')
)

points = base.mark_circle(size=90).encode(
    x='total_investment_musd:Q',
    color=alt.Color('region:N', legend=alt.Legend(title='Region')),
    tooltip=[
        'country:N',
        'region:N',
        alt.Tooltip('total_investment_musd:Q', format=',.1f', title='Investment (M USD)')
    ]
)

(lines + points).properties(width=650, height=22 * len(df_top)).configure_view(stroke=None)


alt.LayerChart(...)

In [156]:
# (We already have plant_level deduped correctly)
plant_level = (
    cofi_df()
    .group_by(["country", "country_iso3c", "region", "location_id.x"])
    .agg(pl.col("total_investment_amount").max().alias("plant_total_investment"))
)

# Bring capacity + fuel info back
plant_level_with_capacity = plant_level.join(
    cofi_df().select(["location_id.x", "installed_capacity"]),
    on="location_id.x",
    how="left"
)

# Combine techs and sum per country
country_scatter = (
    plant_level_with_capacity
    .group_by(["country", "country_iso3c", "region"])
    .agg([
        pl.col("plant_total_investment").sum().alias("total_investment_musd"),
        pl.col("installed_capacity").sum().alias("total_capacity_mw")
    ])
    .sort("total_investment_musd", descending=True)
).to_pandas()


Could not determine dtype for column 39, falling back to string
Could not determine dtype for column 40, falling back to string
Could not determine dtype for column 42, falling back to string
Could not determine dtype for column 39, falling back to string
Could not determine dtype for column 40, falling back to string
Could not determine dtype for column 42, falling back to string


In [157]:
plant_level_with_capacity

country,country_iso3c,region,location_id.x,plant_total_investment,installed_capacity
str,str,str,i64,f64,f64
"""Bulgaria""","""BGR""","""Europe""",1090625,16.1033,5.9
"""Cambodia""","""KHM""","""Asia""",1308792,6.0,60.0
"""Bulgaria""","""BGR""","""Europe""",1092387,null,14.5
"""Egypt""","""EGY""","""Africa""",1119743,86.9,165.5
"""South Korea""","""KOR""","""Asia""",null,null,null
…,…,…,…,…,…
"""Italy""","""ITA""","""Europe""",1102288,null,7.9
"""South Korea""","""KOR""","""Asia""",1138209,null,8.0
"""Italy""","""ITA""","""Europe""",1082853,730.06,27.5


In [172]:
countries_codes = (
    pl.read_csv("data/countries_codes_and_coordinates.csv")
    .with_columns(pl.col("Country").str.strip_chars())
    .rename({"Country": "country"})   # <-- key fix
    .select([
        "country",
        "Alpha-2 code",
        "Alpha-3 code",
        pl.col("Latitude (average)").alias("Latitude"),
        pl.col("Longitude (average)").alias("Longitude"),
    ])
)

result = plant_level_with_capacity.join(
    countries_codes,
    on="country",   # now exists in both
    how="left"
)


In [173]:
result = result.with_columns([
    pl.col("Latitude")
      .str.replace_all('"', '')   # remove quotes
      .str.strip_chars()          # remove whitespace
      .cast(pl.Float64),
    pl.col("Longitude")
      .str.replace_all('"', '')
      .str.strip_chars()
      .cast(pl.Float64)
])

df = result.to_pandas()

df

,country,country_iso3c,region,location_id.x,plant_total_investment,installed_capacity,Alpha-2 code,Alpha-3 code,Latitude,Longitude
0,Bulgaria,BGR,Europe,1090625.0,16.103300,5.90,"""BG""","""BGR""",43.0000,25.0000
1,Cambodia,KHM,Asia,1308792.0,6.000000,60.00,"""KH""","""KHM""",13.0000,105.0000
2,Bulgaria,BGR,Europe,1092387.0,NaN,14.50,"""BG""","""BGR""",43.0000,25.0000
3,Egypt,EGY,Africa,1119743.0,86.900000,165.50,"""EG""","""EGY""",27.0000,30.0000
4,South Korea,KOR,Asia,NaN,NaN,NaN,"""KR""","""KOR""",37.0000,127.5000
...,...,...,...,...,...,...,...,...,...,...
107,Italy,ITA,Europe,1102288.0,NaN,7.90,"""IT""","""ITA""",42.8333,12.8333
108,South Korea,KOR,Asia,1138209.0,NaN,8.00,"""KR""","""KOR""",37.0000,127.5000
109,Italy,ITA,Europe,1082853.0,730.060000,27.50,"""IT""","""ITA""",42.8333,12.8333
110,Thailand,THA,Asia,1104037.0,4.756711,7.46,"""TH""","""THA""",15.0000,100.0000


In [176]:
result.select([
    pl.col("Latitude").min().alias("lat_min"),
    pl.col("Latitude").max().alias("lat_max"),
    pl.col("Longitude").min().alias("lon_min"),
    pl.col("Longitude").max().alias("lon_max"),
])


lat_min,lat_max,lon_min,lon_max
f64,f64,f64,f64
-34.0,60.0,-80.0,127.5


In [178]:
import altair as alt
from vega_datasets import data

df = result.to_pandas()
world = alt.topo_feature(data.world_110m.url, 'countries')

base = (
    alt.Chart(world)
    .mark_geoshape(strokeWidth=0.5)
    .properties(width=900, height=500)
    .project('equalEarth')
)

bubbles = (
    alt.Chart(df)
    .mark_circle(opacity=0.7, stroke='black', strokeWidth=0.4)
    .encode(
        longitude='Longitude:Q',
        latitude='Latitude:Q',
        size=alt.Size('installed_capacity:Q', scale=alt.Scale(range=[20, 2000]), legend=None),
        color=alt.Color('region:N', legend=alt.Legend(title="Region")),
        tooltip=['country:N', 'region:N', 'installed_capacity:Q']
    )
)

bubble_map = base + bubbles
bubble_map


alt.LayerChart(...)

In [ ]:
country_bubble = (
    alt.Chart(country_scatter)
    .mark_circle(opacity=0.7, stroke='black', strokeWidth=0.4)
    .encode(
        x=alt.X('total_capacity_mw:Q',
                title='Total Installed Capacity (MW)',
                scale=alt.Scale(type='sqrt')),
        y=alt.Y('total_investment_musd:Q',
                title='Total Investment (USD Millions)',
                scale=alt.Scale(type='sqrt')),
        size=alt.Size('total_investment_musd:Q',
                      scale=alt.Scale(range=[50, 1500]), legend=None),
        color=alt.Color('region:N', legend=alt.Legend(title="Region")),
        tooltip=[
            alt.Tooltip('country:N'),
            alt.Tooltip('region:N'),
            alt.Tooltip('total_capacity_mw:Q', title='Capacity (MW)', format=','),
            alt.Tooltip('total_investment_musd:Q', title='Investment (M USD)', format=',.1f')
        ]
    )
    .properties(width=750, height=450)
)

country_bubble


alt.Chart(...)

In [142]:
china_solar_exports_countries_df().write_csv("./data/china_solar_exports_countries_df.csv")

In [146]:
def china_solar_exports_countries_totals():
    return (
        china_solar_exports_countries_df()
        .group_by(["Area", "Commodity type"])
        .agg(pl.col("Capacity (MW)").sum().alias("Total Capacity (MW)"))
        .sort("Total Capacity (MW)", descending=True)
    )

top10 = (
    china_solar_exports_countries_totals()
    .group_by("Area")
    .agg(pl.col("Total Capacity (MW)").sum().alias("Country Total"))
    .sort("Country Total", descending=True)
    .head(20)
    .select("Area")
)

df_top10 = (
    china_solar_exports_countries_totals()
    .join(top10, on="Area", how="inner")
    .to_pandas()
)

In [147]:
df_top10

,Area,Commodity type,Total Capacity (MW)
0,Netherlands,Panels,195607.38
1,Brazil,Panels,91933.42
2,India,Panels,91833.42
3,India,Cells,72246.86
4,Japan,Panels,56766.59
5,Spain,Panels,51540.88
6,Pakistan,Panels,49724.14
7,Vietnam,Wafers,45969.82
8,Thailand,Wafers,43682.15
9,Australia,Panels,42908.91


In [73]:
chart = (
    alt.Chart(df_top10)
    .mark_bar()
    .encode(
        x=alt.X("Area:N", sort="-y", title=None),
        y=alt.Y("Total Capacity (MW):Q", title="Total Capacity (MW)"),
        color=alt.Color("Commodity type:N", title="Technology"),
        tooltip=[
            alt.Tooltip("Area:N", title="Country"),
            alt.Tooltip("Commodity type:N", title="Technology"),
            alt.Tooltip("Total Capacity (MW):Q", title="Capacity (MW)", format=",")
        ]
    )
    .properties(width=700, height=400)
)

chart


alt.Chart(...)

In [118]:
uae_monthly_total = (
    china_solar_exports_countries_df()
    .filter(pl.col("Area") == "United Arab Emirates")     # adjust to "UAE" if needed
    .group_by("Date")
    .agg(pl.col("Capacity (MW)").sum().alias("Total Capacity (MW)"))
    .sort("Date")
)


In [119]:


uae_chart = (
    alt.Chart(uae_monthly_total.to_pandas())
    .mark_line(point=True, strokeWidth=2)
    .encode(
        x=alt.X(
            'yearmonth(Date):T',
            title='Date',
            axis=alt.Axis(format='%Y', tickCount='year')  # show one tick per year
        ),
        y=alt.Y(
            'Total Capacity (MW):Q',
            title='Total Exported Capacity (MW)'
        ),
        tooltip=[
            alt.Tooltip('yearmonth(Date):T', title='Month'),
            alt.Tooltip('Total Capacity (MW):Q', title='Capacity (MW)', format=',')
        ]
    )
    .properties(
        width=750,
        height=400,
        title="Chinese Solar Exports to the United Arab Emirates Over Time"
    )
)

uae_chart


alt.Chart(...)

In [81]:
top10_countries = (
    china_solar_exports_countries_df()
    .group_by("Area")
    .agg(pl.col("Capacity (MW)").sum().alias("Total Capacity (MW)"))
    .sort("Total Capacity (MW)", descending=True)
    .head(5)
    .select("Area")
)


top10_timeseries = (
    china_solar_exports_countries_df()
    .join(top10_countries, on="Area", how="inner")   # keep only top 10 countries
    .group_by(["Area", "Date"])
    .agg(pl.col("Capacity (MW)").sum().alias("Total Capacity (MW)"))
    .sort(["Area", "Date"])
)

df_top10_ts = top10_timeseries.to_pandas()


In [148]:
top10_timeseries

Area,Date,Total Capacity (MW)
str,date,f64
"""Brazil""",2017-01-01,119.1
"""Brazil""",2017-02-01,121.67
"""Brazil""",2017-03-01,148.46
"""Brazil""",2017-04-01,66.04
"""Brazil""",2017-05-01,59.73
…,…,…
"""Viet Nam""",2025-04-01,2069.72
"""Viet Nam""",2025-05-01,1520.86
"""Viet Nam""",2025-06-01,1635.01


In [83]:


chart_top10 = (
    alt.Chart(df_top10_ts)
    .mark_line(point=True, strokeWidth=2)
    .encode(
        x=alt.X(
            'yearmonth(Date):T',
            title='Date',
            axis=alt.Axis(format='%Y', tickCount='year')
        ),
        y=alt.Y(
            'Total Capacity (MW):Q',
            title='Total Exported Capacity (MW)'
        ),
        color=alt.Color(
            'Area:N',
            title='Country',
            legend=alt.Legend(columns=2, orient="right")
        ),
        tooltip=[
            alt.Tooltip('Area:N', title='Country'),
            alt.Tooltip('yearmonth(Date):T', title='Month'),
            alt.Tooltip('Total Capacity (MW):Q', title='Capacity (MW)', format=',')
        ]
    )
    .properties(
        width=850,
        height=450,
        title="Chinese Solar Exports to the Top 10 Destination Countries Over Time"
    )
)

chart_top10


alt.Chart(...)

In [91]:
df_6m = (
    china_solar_exports_countries_df()
    .filter(pl.col("Date") >= pl.date(2017, 1, 1))
    .group_by_dynamic(
        index_column="Date",
        every="6mo",
        period="6mo",
        by="Area"
    )
    .agg(pl.col("Capacity (MW)").sum().alias("Total Capacity (MW)"))
    .sort(["Area", "Date"])
)


/var/folders/xw/t_4hhz1d5plg9hglf0dg5m200000gn/T/ipykernel_16380/1337482591.py:4: DeprecationWarning: the argument `by` for `DataFrame.group_by_dynamic` is deprecated. It was renamed to `group_by` in version 0.20.14.
  .group_by_dynamic(


In [92]:
top10 = (
    df_6m.group_by("Area")
         .agg(pl.col("Total Capacity (MW)").sum().alias("Total Country Capacity (MW)"))
         .sort("Total Country Capacity (MW)", descending=True)
         .head(5)
         .select("Area")
)

df_6m_top10 = df_6m.join(top10, on="Area", how="inner")
df_6m_pd = df_6m_top10.to_pandas()


In [ ]:
top5_2024 = (
    china_solar_exports_countries_df()
    .filter(pl.col("Date").dt.year() == 2017)   # extract year from date
    .group_by("Area")
    .agg(pl.col("Capacity (MW)").sum().alias("Total Capacity (MW)"))
    .sort("Total Capacity (MW)", descending=True)
    .head(5)
)

top5_2024

Area,Total Capacity (MW)
str,f64
"""Netherlands""",41419.93
"""Brazil""",17790.71
"""India""",14479.96
"""Viet Nam""",11443.05
"""Malaysia""",11213.49


In [136]:
def top_countries_by_tech(year: int, commodity: str, n: int = 5):
    """
    Returns the top N countries by total exported capacity for a given technology and year.
    
    Parameters
    ----------
    year : int
        The year to filter on (e.g., 2017).
    commodity : str
        The technology name in the 'Commodity type' column.
    n : int, optional
        How many top countries to return (default = 5).
    """
    return (
        china_solar_exports_countries_df()
        .filter(
            (pl.col("Date").dt.year() == year)
            & (pl.col("Commodity type") == commodity)
        )
        .group_by("Area")
        .agg(pl.col("Capacity (MW)").sum().alias("Total Capacity (MW)"))
        .sort("Total Capacity (MW)", descending=True)
        .head(n)
    )

# Example:
top_countries_by_tech(2024, "Panels") #Cells, Panels, Wafers


Area,Total Capacity (MW)
str,f64
"""Netherlands""",39190.57
"""Brazil""",21924.45
"""Pakistan""",16384.51
"""Saudi Arabia""",16128.69
"""India""",15474.47


In [145]:
def top_countries_by_tech_total(commodity: str, n: int = 5):
    """
    Returns the top N countries by total exported capacity for a given technology,
    aggregated across all years.

    Parameters
    ----------
    commodity : str
        The technology name in the 'Commodity type' column.
    n : int, optional
        How many top countries to return (default = 5).
    """
    return (
        china_solar_exports_countries_df()
        .filter(pl.col("Commodity type") == commodity)
        .group_by("Area")
        .agg(pl.col("Capacity (MW)").sum().alias("Total Capacity (MW)"))
        .sort("Total Capacity (MW)", descending=True)
        .head(n)
    )

# Example:
top_countries_by_tech_total("Wafers")


Area,Total Capacity (MW)
str,f64
"""Vietnam""",45969.82
"""Thailand""",43682.15
"""India""",33945.23
"""Malaysia""",31737.99
"""South Korea""",22135.87


In [183]:
pv_module_manufacture_share_2023=pl.read_excel("data/pv_manufacture_share_2023.xlsx")
pv_module_manufacture_share_2023

country,share
str,f64
"""China""",84.6
"""Vietnam""",3.4
"""India""",2.7
"""Thailand""",2.3
"""United States""",2.2
"""Malaysia""",2.1
"""Others""",2.6


In [184]:
base = alt.Chart(pv_module_manufacture_share_2023).encode(
    alt.Theta("share:Q").stack(True),
    alt.Color("country:N").legend(None)
)

pie = base.mark_arc(outerRadius=120)
text = base.mark_text(radius=140, size=20).encode(text="country:N")

pie + text

alt.LayerChart(...)